<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/(Not_Suitable)_Map_Codes_General.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Read into one dataset and get its indexes

Note: get the exact file name to read

In [1]:
import chardet
with open('DOMComprasCardio.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'confidence': 0.99, 'encoding': 'utf-8', 'language': ''}

In [2]:
import pandas as pd
OrgData = pd.read_csv('DOMComprasCardio.csv',encoding='utf-8')
OrgData.head(1)

,Unnamed: 0,CODIGO_UNIDAD_COMPRA,UNIDAD_COMPRA,CODIGO_PROCESO,MODALIDAD,ESTADO_PROCESO,FECHA_PUBLICACION,ID_ARTICULO,SUBCLASE_UNSPSC,DESCRIPCION_SUBCLASE,DESCRIPCION_ARTICULO,CANTIDAD,ABR_UNIDAD_MEDIDA,DESCRIPCION_UNIDAD_MEDIDA,PRECIO_UNITARIO_ESTIMADO,MONTO_TOTAL_ESTIMADO,CUENTA_PRESUPUESTARIA
0,1,195,Sistema Unico de Beneficiarios (SIUBEN),SIUBEN-DAF-CM-2019-0013,Compras Menores,Proceso adjudicado y celebrado,2019-03-13 15:00:06,12462773,51121709,Carvedilol,"Carvedilol 25 MG, 30/1",5.0,CAJ,Caja,450,2250.0,2.3.4.1.01


Note: get the special column name to get the dirty names

In [3]:
OrgDataDirtyName = OrgData['DESCRIPCION_ARTICULO']

In [4]:
len(OrgDataDirtyName) # length of drug names

3077

In [5]:
OrgDataDirtyName = list(set(OrgDataDirtyName))
len(OrgDataDirtyName) # length of drug names after remove duplicates

1959

In [6]:
# uppercase everything
OrgDataDirtyNameU = []
for i in OrgDataDirtyName:
  OrgDataDirtyNameU.append(i.upper())

In [7]:
OrgDataDirtyNameU[:5]

['CAPTOPRIL\t25MG TAB',
 'CARVEDILOL 12.5 MG',
 'ATORVASTATINA 20MG CAJA DE 10 TABLETAS',
 'IRBESARTAN 300 MG + AMLODIPINA 10 MG X 30 TAB',
 'ATENOLOL 100 MG / V.O TABLETAS']

translate from Spainish to English

In [8]:
!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
example = OrgDataDirtyNameU[0]
translation = translator.translate(example, dest='en')
print(translation.text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 1.4 MB 10.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=eb4fa59178d5b146bb2b77d83fab29091d9af68ce98f8ae2cf9bb78ce1dd2083
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
CAPTOPRIL	25MG TAB


In [9]:
OrgDataDirtyNameUEng = []
for item in OrgDataDirtyNameU:
  translation = translator.translate(item, dest='en')
  OrgDataDirtyNameUEng.append(translation.text)
# runs 1 mins

In [10]:
OrgDataDirtyNameUEng[:5]

['CAPTOPRIL\t25MG TAB',
 'CARVEDILOL 12.5 MG',
 'ATORVASTATIN 20MG BOX OF 10 TABLETS',
 'IRBESARTAN 300 MG + AMLODIPINA 10 MG X 30 TAB',
 'ATENOLOL 100 MG / V.O TABLETS']

In [11]:
# get the orignila index about medicine name for each drug name 

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

OrgDataDirtyNameUESpl = []
for item in OrgDataDirtyNameUEng:
  tokenlist = tokenizer.tokenize(item)
  OrgDataDirtyNameUESpl.append(tokenlist)

In [12]:
OrgDataDirtyNameUESpl[:1]

[['CAPTOPRIL', '25MG', 'TAB']]

In [13]:
OrgDataDirtyNameIndex  = []
for item in OrgDataDirtyNameUESpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  OrgDataDirtyNameIndex.append(templist)

In [14]:
OrgDataDirtyNameIndex[:5]

[['CAPTOPRIL'],
 ['CARVEDILOL'],
 ['ATORVASTATIN', 'TABLETS'],
 ['IRBESARTAN', 'AMLODIPINA'],
 ['ATENOLOL', 'TABLETS']]

In [15]:
# delete the information in each piece of drug name after SIMILAR
def DelSIMILAR(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'SIMILAR':
      finalindex = subindex
  return subitem[:finalindex]

In [16]:
# delete the information in each piece of drug name after EQUIVALENT
def DelEQUIVALENT(subitem):
  finalindex = len(subitem)
  for subindex in range(0, len(subitem)):
    if subitem[subindex] == 'EQUIVALENT':
      finalindex = subindex
  return subitem[:finalindex]

In [17]:
OrgDataDirtyNameIndexDel = []
for item in OrgDataDirtyNameIndex:
  add = DelSIMILAR(item)
  add = DelEQUIVALENT(add)
  OrgDataDirtyNameIndexDel.append(add)

In [18]:
OrgDataDirtyNameIndexDel[:5]

[['CAPTOPRIL'],
 ['CARVEDILOL'],
 ['ATORVASTATIN', 'TABLETS'],
 ['IRBESARTAN', 'AMLODIPINA'],
 ['ATENOLOL', 'TABLETS']]

## Part 2: Set up indexes for clean catagol drug names

In [19]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [20]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [21]:
CatalogNameUS[:5] # after split by ","

[['CARBOPLATIN\xa0150\xa0MG',
  '\xa0POWDER\xa0FOR\xa0INJECTION',
  '\xa015\xa0ML\xa0VIAL',
  '\xa01X1'],
 ['NIFEDIPINE 10 MG', ' TABLET', ' TBD'],
 ['SITAGLIPTIN 50 MG + METFORMIN 1000 MG', ' TABLET', ' TBD'],
 ['METFORMIN 1000 MG + GLIMEPIRIDE 4 MG', ' TABLET', ' TBD'],
 ['AMLODIPINE 10 MG + VALSARTAN 160 MG', ' TBD', ' TBD']]

In [22]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])

In [23]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [24]:
PresentationForm[:5]# the presentation form

[' SOLUTION FOR INFUSION',
 ' POWDER FOR INJECTION. 1000MG ROAD BOTTLE.',
 ' INJECTALBE SOLUTION',
 ' TABLET (CHEWABLE)',
 ' CAPSULE']

In [25]:
PresentationDiction = []

from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
tokenizer = WordPunctTokenizer()

for item in PresentationForm:
  tokenlist = tokenizer.tokenize(item)
  PresentationDiction.append(tokenlist)

In [26]:
PresentationDiction2 = []

for item in PresentationDiction:
  for subitem in item:
    if len(subitem) > 3:
      PresentationDiction2.append(subitem)

In [27]:
PresentationDiction2 = list(set(PresentationDiction2))

In [28]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [29]:
print(len(MainMedicine), MainMedicine[:5])

462 ['CARBOPLATIN\xa0150\xa0MG', 'NIFEDIPINE 10 MG', 'SITAGLIPTIN 50 MG + METFORMIN 1000 MG', 'METFORMIN 1000 MG + GLIMEPIRIDE 4 MG', 'AMLODIPINE 10 MG + VALSARTAN 160 MG']


In [30]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [31]:
MainMedicineSpl[:5]

[['CARBOPLATIN', '150', 'MG'],
 ['NIFEDIPINE', '10', 'MG'],
 ['SITAGLIPTIN', '50', 'MG', '+', 'METFORMIN', '1000', 'MG'],
 ['METFORMIN', '1000', 'MG', '+', 'GLIMEPIRIDE', '4', 'MG'],
 ['AMLODIPINE', '10', 'MG', '+', 'VALSARTAN', '160', 'MG']]

In [32]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  MainMedicineIndex.append(templist)

In [33]:
MainMedicineIndex[:5]

[['CARBOPLATIN'],
 ['NIFEDIPINE'],
 ['SITAGLIPTIN', 'METFORMIN'],
 ['METFORMIN', 'GLIMEPIRIDE'],
 ['AMLODIPINE', 'VALSARTAN']]

In [34]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [35]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [36]:
len(CatalogIndexData_df)

462

In [37]:
CatalogIndexData_df[:5]

,Catalog Name,Catalog Index
0,"CARBOPLATIN 150 MG, POWDER FOR INJECTION, 15 M...",[CARBOPLATIN]
1,"NIFEDIPINE 10 MG, TABLET, TBD",[NIFEDIPINE]
2,"SITAGLIPTIN 50 MG + METFORMIN 1000 MG, TABLET,...","[SITAGLIPTIN, METFORMIN]"
3,"METFORMIN 1000 MG + GLIMEPIRIDE 4 MG, TABLET, TBD","[METFORMIN, GLIMEPIRIDE]"
4,"AMLODIPINE 10 MG + VALSARTAN 160 MG, TBD, TBD","[AMLODIPINE, VALSARTAN]"


In [38]:
# create a list for main medicine from catalog name data set
CatalogMedName = []
for item in MainMedicineIndex:
  for subitem in item:
    CatalogMedName.append(subitem)

In [39]:
CatalogMedName = list(set(CatalogMedName))
len(CatalogMedName)

206

In [40]:
CatalogMedName[:3]

['BEVACIZUMAB', 'AMOXICILLIN', 'ENTECAVIR']

## Part 3: Write the Index Data Set According to the Catalog Name Index

In [41]:
# for the vocabulaies which are included in the medicine index get from clean data set, keep them in the 
# Chile 2 data index
def KeepMetAnd(subitem):
  addlist = []
  for element in subitem:
    if element in CatalogMedName:
      addlist.append(element)
  return addlist

In [42]:
DatafinIndex = []
for sublist in OrgDataDirtyNameIndexDel:
  add = KeepMetAnd(sublist)
  DatafinIndex.append(add)

In [43]:
# count the number of Chile index sub list which is null []
count = 0
for i in DatafinIndex:
  if not i:
    count += 1

the length of total drug names and null index

In [44]:
print(len(DatafinIndex), count) 

1959 464


In [45]:
nullindexlist = []
for i in range(0, len(DatafinIndex)):
  if not DatafinIndex[i]:
    nullindexlist.append(i)

In [46]:
!pip install -U spacy
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.5 MB/s 
     |████████████████████████████████| 6.0 MB 9.9 MB/s 
     |████████████████████████████████| 13.5 MB 38.4 MB/s 
     |████████████████████████████████| 71 kB 8.6 MB/s 
     |████████████████████████████████| 188 kB 56.0 MB/s 
     |████████████████████████████████| 10.1 MB 35.5 MB/s 
     |████████████████████████████████| 660 kB 58.7 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.1
    Uninstalling pydantic-1.9.1:
      Successfully uninstalled pydantic-1.9.1
  Attempting uninstall: thinc
    Found exi

In [47]:
## Install scispaCy models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_craft_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_jnlpba_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.6 MB 1.2 MB/s 
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.4.0-py3-none-any.whl size=15660350 sha256=444c756ddcd45e54552a1f9d57f55d11431412c61de44244d913e3b4bfe8c9fc
  Stored in directory: /root/.cache/pip/wheels/31/4b/3a/03f2732eb69789a7df3bf29c1bc010f660ada5f3c295d4073c
Successfully built en-core-sci-sm
  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scisp

In [48]:
import scispacy
import spacy

#Core models
import en_core_sci_sm
import en_core_sci_lg

#NER specific models
import en_ner_craft_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_bionlp13cg_md

#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

In [49]:
#Load specific model and pass text through
nlp = en_core_sci_lg.load()

use spacy nlp to get informations from the drug names which with null index

In [50]:
DatadrugOutDataName = []
for nullindex in nullindexlist:
  tempdrugname = OrgDataDirtyNameIndexDel[nullindex]
  tempdrugname_str = str(' '.join(tempdrugname))
  doc = nlp(tempdrugname_str)
  temptext = doc.text
  DatadrugOutDataName.append(temptext)

In [51]:
drugOutDataNameNullNumber = 0

for i in DatadrugOutDataName:
  if not i:
    drugOutDataNameNullNumber += 1

the number of drug name which are still null

In [52]:
drugOutDataNameNullNumber

0

In [53]:
# for these null element in list, use Chile1drugOutDataName to complete them
j = 0
for i in nullindexlist:
  DatafinIndex[i] = DatadrugOutDataName[j]
  j += 1

In [54]:
for index in range(len(DatafinIndex)):
  if not DatafinIndex[index]:
    DatafinIndex[index] = OrgDataDirtyNameIndexDel[index]

In [55]:
nullindex = []
for index in range(len(DatafinIndex)):
  if not DatafinIndex[index]:
    nullindex.append(index)

In [56]:
nullindex

[]

In [57]:
PreList = []

for item in OrgDataDirtyNameIndexDel:
  addlist = []
  for subitem in item:
    if subitem in PresentationDiction2:
      addlist.append(subitem)
  PreList.append(list(set(addlist)))

In [58]:
WeightList = []

for item in OrgDataDirtyNameIndexDel:
  addlist = []
  for subitem in range(0, len(item)):
    if item[subitem].isdigit():
      if subitem + 1 < len(item):
        if item[subitem + 1].lower() in ['mg', 'ml']:
          addlist.append(item[subitem])
          addlist.append(item[subitem + 1])
  WeightList.append(list(set(addlist)))

In [59]:
print(len(DatafinIndex), len(PreList), len(WeightList))

1959 1959 1959


In [60]:
# for catalog name index dataframe, transform list to string
str_DatafinIndex = []
for item in DatafinIndex:
  if not isinstance(item, str):
    addsub = []
    for subitem in item:
      if subitem:
        addsub.append(subitem)
        newitem = ' '.join(addsub)
  else:
    newitem = item

  str_DatafinIndex.append(newitem)

In [61]:
str_DatafinIndex[:4]

['CAPTOPRIL', 'CARVEDILOL', 'ATORVASTATIN', 'IRBESARTAN']

In [62]:
str_PreList = []

for item in PreList:
  newitem = ' '.join(item)
  str_PreList.append(newitem)

In [63]:
str_WeightList = []

for item in WeightList:
  newitem = ' '.join(item)
  str_WeightList.append(newitem)

In [64]:
DrugDataWrite = {"DOMComprasCardio Dirty Name English": OrgDataDirtyNameUEng, "Final Drug Index": str_DatafinIndex, "Presentation Form": str_PreList, "Weight": str_WeightList}


In [65]:
DrugDataWrite_df = pd.DataFrame(DrugDataWrite)

In [66]:
DrugDataWrite_df

,DOMComprasCardio Dirty Name English,Final Drug Index,Presentation Form,Weight
0,CAPTOPRIL\t25MG TAB,CAPTOPRIL,,
1,CARVEDILOL 12.5 MG,CARVEDILOL,,
2,ATORVASTATIN 20MG BOX OF 10 TABLETS,ATORVASTATIN,,
3,IRBESARTAN 300 MG + AMLODIPINA 10 MG X 30 TAB,IRBESARTAN,,
4,ATENOLOL 100 MG / V.O TABLETS,ATENOLOL,,
...,...,...,...,...
1954,LOSARTAN 100MG ALFA,LOSARTAN,,
1955,METILDOPA 250 MG,METILDOPA,,
1956,ATORVASTATIN 10MG BOX OF 10 SUED TABLETS,ATORVASTATIN,,
1957,CAPTOPRIL,CAPTOPRIL,,


In [67]:
DrugDataWrite_df.to_excel("DOMComprasCardio (Write).xlsx", sheet_name = 'DOMComprasCardio')  

## Part 4: Use Final Drug Index Map With Catalog Name Index

In [68]:
!pip install simhash

from simhash import Simhash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
PairIndexData = {"drug names": OrgDataDirtyNameUEng, "Catalog Index": str_DatafinIndex}

In [70]:
PairIndexData_df = pd.DataFrame(PairIndexData)

In [71]:
PairIndexData_df

,drug names,Catalog Index
0,CAPTOPRIL\t25MG TAB,CAPTOPRIL
1,CARVEDILOL 12.5 MG,CARVEDILOL
2,ATORVASTATIN 20MG BOX OF 10 TABLETS,ATORVASTATIN
3,IRBESARTAN 300 MG + AMLODIPINA 10 MG X 30 TAB,IRBESARTAN
4,ATENOLOL 100 MG / V.O TABLETS,ATENOLOL
...,...,...
1954,LOSARTAN 100MG ALFA,LOSARTAN
1955,METILDOPA 250 MG,METILDOPA
1956,ATORVASTATIN 10MG BOX OF 10 SUED TABLETS,ATORVASTATIN
1957,CAPTOPRIL,CAPTOPRIL


## Part 5: combine the drug name index pair data frame and full catagol name index data frame to get the result

In [72]:
pairIndexlist = []
for dirtyname in str_DatafinIndex:
  distance_list = []
  SubMedicineList = []
  for index in MainMedicineIndex:
    add = False
    for item in index:
      for dirtyitem in dirtyname:
        if dirtyitem == item:
          add = True
    if add == True:
      SubMedicineList.append(index)
  if not SubMedicineList:
    SubMedicineList 
  for index in MainMedicineIndex:
    distance = Simhash(index).distance(Simhash(dirtyname))
    distance_list.append(distance)
  index_min = min(range(len(distance_list)), key=distance_list.__getitem__) # get the index which correspond to the minimum distance
  add_index = MainMedicineIndex[index_min]
  pairIndexlist.append(add_index)

In [73]:
CatPairIndexData = {"drug names": OrgDataDirtyNameUEng, "Catalog Index": str_DatafinIndex, "Paired Catalog": pairIndexlist}

In [75]:
CatPairIndexData_df = pd.DataFrame(CatPairIndexData)

In [76]:
CatPairIndexData_df

,drug names,Catalog Index,Paired Catalog
0,CAPTOPRIL\t25MG TAB,CAPTOPRIL,"[INSULIN, GLULISINE]"
1,CARVEDILOL 12.5 MG,CARVEDILOL,[DACTINOMICNA]
2,ATORVASTATIN 20MG BOX OF 10 TABLETS,ATORVASTATIN,"[AMLODIPINE, CANDESARTAN, HYDROCHLOROTIAZIDE]"
3,IRBESARTAN 300 MG + AMLODIPINA 10 MG X 30 TAB,IRBESARTAN,[METHOTREXATE]
4,ATENOLOL 100 MG / V.O TABLETS,ATENOLOL,"[DOCETAXEL, SOLVENT]"
...,...,...,...
1954,LOSARTAN 100MG ALFA,LOSARTAN,"[METHOTREXATE, SODIUM]"
1955,METILDOPA 250 MG,METILDOPA,"[CHLORTHALIDONE, VALSARTAN]"
1956,ATORVASTATIN 10MG BOX OF 10 SUED TABLETS,ATORVASTATIN,"[AMLODIPINE, CANDESARTAN, HYDROCHLOROTIAZIDE]"
1957,CAPTOPRIL,CAPTOPRIL,"[INSULIN, GLULISINE]"


In [82]:
# for catalog name index dataframe, transform list to string
str_MainMedicineIndex = []
for item in MainMedicineIndex:
  newitem = ' '.join(item)
  str_MainMedicineIndex.append(newitem)

In [83]:
newCatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": str_MainMedicineIndex}

In [84]:
newCatalogIndexData_df = pd.DataFrame(newCatalogIndexData)

In [85]:
newCatalogIndexData_df[:2]

,Catalog Name,Catalog Index
0,"SAXAGLIPTIN 5 MG, TABLET, TBD",SAXAGLIPTIN
1,"LEVOTHYROXINE 0.625 MG, TABLET, TBD",LEVOTHYROXINE


In [86]:
combine_pair_df = pd.merge(newPairIndexData_df, newCatalogIndexData_df, on='Catalog Index', how='left')

In [87]:
#combine_pair_df.to_excel("combine_pair_df_chile_1.xlsx", sheet_name = 'combine_pair_df chile 1')  

## Part 6: Combine the original drug name in Dirty Data Set with the pairing data frame we have got, then select for the catalog name second time (compute the distance between the original drug name and the sub set of catalog under each name in our result)

In [88]:
OrgDataDirtyName[:5]

['IBERSARTAN 150MG TAB.',
 'LOSARTAN 100MG TABLETA',
 'Amlodipin de 5 mg (caja de 30 tabletas)',
 'Rosuvastatina 40mg caja de 10 Tabletas Mamey',
 'Captopril 50 MG Tab']

In [89]:
str_DatafinIndex[:5]

['IRBESARTAN',
 'LOSARTAN',
 'AMLODIPIN',
 'ROSUVASTATINE TABLETS MAMEY',
 'CAPTOPRIL']

In [90]:
print(len(OrgDataDirtyName), len(str_DatafinIndex))

1959 1959


In [91]:
# translate the original full drug name into english

OrgDataDirtyNameEng = []

for item in OrgDataDirtyName:
  temp = translator.translate(item, dest='en')
  OrgDataDirtyNameEng.append(temp.text)

# this block runs 7 mins

In [92]:
# uppercase everything
OrgDataDirtyNameEngU = []
for i in OrgDataDirtyNameEng:
  OrgDataDirtyNameEngU.append(i.upper())

OrgDataDirtyNameEngU[:5]

['IRBESARTAN 150 MG TAB.',
 'LOSARTAN 100 MG TABLET',
 'AMLODIPIN 5 MG (BOX OF 30 TABLETS)',
 'ROSUVASTATIN 40MG BOX OF 10 TABLETS MAMEY',
 'CAPTOPRIL 50 MG TAB']

In [93]:
# combine uppercase english chile 1 drug name with its indexes
drugfn_index = {"drugs english uppercase names": OrgDataDirtyNameEngU, "drug names": str_DatafinIndex}

In [94]:
drugfn_index_df = pd.DataFrame(drugfn_index)

In [95]:
drugfn_index_df[:2]

,drugs english uppercase names,drug names
0,IRBESARTAN 150 MG TAB.,IRBESARTAN
1,LOSARTAN 100 MG TABLET,LOSARTAN


In [96]:
# combine Chile drug original full name, its index with the result data frame (pair with catalog name)

drugs = pd.merge(drugfn_index_df, combine_pair_df, on='drug names', how='left')

In [97]:
drugs[:5]

,drugs english uppercase names,drug names,Catalog Index,Catalog Name
0,IRBESARTAN 150 MG TAB.,IRBESARTAN,METHOTREXATE,"METHOTREXATE 25 MG, TBD, TBD"
1,IRBESARTAN 150 MG TAB.,IRBESARTAN,METHOTREXATE,"METHOTREXATE 10 MG, TBD, TBD"
2,IRBESARTAN 150 MG TAB.,IRBESARTAN,METHOTREXATE,"METHOTREXATE 15 MG, TBD, TBD"
3,IRBESARTAN 150 MG TAB.,IRBESARTAN,METHOTREXATE,"METHOTREXATE 20 MG, TBD, TBD"
4,IRBESARTAN 150 MG TAB.,IRBESARTAN,METHOTREXATE,"METHOTREXATE 25 MG, TBD, TBD"


In [98]:
print(len(drugs), len(set(drugs['drugs english uppercase names'])))

316962 1675


In [104]:
# get the list that contains first pairing names list for each original drug full name
fistpairinglist = []
i = 0
for name in OrgDataDirtyNameEngU:
  subpairlist = []
  while i < len(drugs):
    if drugs['drugs english uppercase names'][i] == name:
      subpairlist.append(drugs['Catalog Name'][i])
      i += 1
    else:
      break
  fistpairinglist.append(subpairlist)
# runs 4 mins

In [107]:
print(len(fistpairinglist), len(OrgDataDirtyNameUEng))

1959 1959


In [108]:
finalpairname = []
for item in range(len(OrgDataDirtyNameUEng)):
  distance_list = []
  for pairname in fistpairinglist[item]:
    distance = Simhash(OrgDataDirtyNameUEng[item]).distance(Simhash(pairname))
    distance_list.append(distance)
  index_min = min(range(len(distance_list)), key=distance_list.__getitem__)
  add_index = fistpairinglist[item][index_min]
  finalpairname.append(add_index)

ValueError: ignored

In [ ]:
# combine the final paired name with the Chile 1 drug english name
Datadrugfn_index = {"drugs english uppercase names": OrgDataDirtyNameUEng, "Final paired catalog name": finalpairname}

In [ ]:
Datadrugfn_index_df = pd.DataFrame(Datadrugfn_index)

In [ ]:
Datadrugfn_index_df[:5]

In [ ]:
Datadrugfn_index_df.to_excel("drugfn_index.xlsx", sheet_name = 'drugfn_index chile 1')  